In [2]:
import requests
import pandas as pd
import yaml
import xml.etree.ElementTree as et
from lxml import etree
import xml.dom.minidom as md

In [29]:
# get the all the authors and their institutions
paper_url='https://api.openalex.org/works/https://doi.org/{DOI}'
author_url = "https://api.openalex.org/authors/{ID}"

years = range(16,23)
results = pd.DataFrame(columns=['doi','Name','Name_Variants','Institution'])
author_ids = []

for year in years:
    for doi in pd.read_csv("G"+str(year)+".csv")["doi"]:   
        response = requests.get(url = paper_url.replace("{DOI}", doi))
        if response.status_code != 200:
            print("not response")
            break
        for author in response.json()['authorships']:
            author_ids.append(author['author']['id'].replace("https://openalex.org/",""))
            r = requests.get(url = author_url.replace("{ID}", author['author']['id'].replace("https://openalex.org/",""))).json()
            # names.append(r['display_name'])
            # inst.append(r['last_known_institution']['display_name'])
            results.loc[len(results.index)] = [doi, r['display_name'], r['display_name_alternatives'], r['last_known_institution']['display_name'] if r['last_known_institution'] else '']
    print(year)    
    
results.to_csv('author_detail.csv', index=False)

16
17
18
19
20
21
22


In [84]:
# set a code to all institutions
author_detail = pd.read_csv('author_detail.csv')
inst_code = pd.read_csv('inst_code.csv')

In [81]:
author_detail['code'] = ['0000' if pd.isna(item) else '{:04d}'.format(inst_code[inst_code['Institution'] == item]['code'].item()) if item else '' for item in author_detail['Institution']]
author_detail.to_csv('author_detail.csv', index=False)

# manually edited in this step. removed duplicates ...

In [187]:
# update xml files with unified authors (name variants corrected)
author_detail = pd.read_csv('author_detail.csv')
xml_folder = '../data/xml/'
years = range(2016,2023)

for idx, year in enumerate(years):
    tree = et.parse(xml_folder+"G"+str(year)[-2:]+".xml")
    root = tree.getroot()
    for paper in root.findall(".//paper"):
        for author in paper.findall(".//author"):
            paper.remove(author)

        for author in author_detail[author_detail['doi'] == paper.find('.//doi').text]['Name'].iloc[::-1]:
            element = et.Element("author")            
            tag_subelement = et.SubElement(element, "first")
            tag_subelement.text = author.rsplit(' ',1)[0]
            tag_subelement = et.SubElement(element, "last")
            tag_subelement.text = author.rsplit(' ',1)[1]
            paper.insert(1,element)
            

    tree = et.ElementTree(root)
    file_name = "G"+str(year)[-2:]+".xml"   
    with open(file_name, "w", encoding="UTF-8") as f:
        f.write(etree.tostring(etree.XML(et.tostring(root, encoding="UTF-8", xml_declaration=True), parser=etree.XMLParser(remove_blank_text=True))).decode())

    xml_pretty_str = md.parse(file_name)
    xml_pretty_str = xml_pretty_str.toprettyxml(encoding='UTF-8').decode()
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(xml_pretty_str)  

    # print(et.tostring(root, encoding='utf8').decode('utf8'))            


In [10]:
# update name_variants.yaml
author_detail = pd.read_csv('author_detail.csv')

dict_file = []

for idx, item in author_detail[['Name', 'code']].drop_duplicates().iterrows():
    dict_file.append({'canonical' : {'first': item['Name'].rsplit(' ',1)[0], 'last': item['Name'].rsplit(' ',1)[1]}, 'id':item['Name'].rsplit(' ',1)[0].replace('.', '')+'-'+item['Name'].rsplit(' ',1)[1].replace('.', ''), 'comment':'uni{:04d}'.format(item['code'])})


with open(r'name_variants.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file, default_flow_style=None)

In [11]:
# produce a line of code for universities
# put it in the second line of hugo\layouts\index.html

inst_code = pd.read_csv('inst_code.csv')

In [23]:
inst_code

,Institution,code
0,University of Saskatchewan,1
1,University of Waterloo,2
2,McMaster University,3
3,Environment and Climate Change Canada,4
4,University of Guelph,5
...,...,...
1489,Center for Excellence in Brain Science and Int...,1490
1490,Office of Oceanic and Atmospheric Research,1491
1491,Deakin University,1492
1492,Osaka Prefecture University,1493


In [31]:
inst_code 
out_str = '{{ $universities := dict'
for idx, inst in inst_code.iloc[:50].iterrows():
    out_str += ' "'+'uni{:04d}'.format(inst['code'])+'" "'+inst['Institution']+'"'    
out_str += '}}' 

out_str

'{{ $universities := dict "uni0001" "University of Saskatchewan" "uni0002" "University of Waterloo" "uni0003" "McMaster University" "uni0004" "Environment and Climate Change Canada" "uni0005" "University of Guelph" "uni0006" "Global Institute for Water Security" "uni0007" "Wilfrid Laurier University" "uni0008" "University of Toronto" "uni0009" "University of Alberta" "uni0010" "Wageningen University & Research" "uni0011" "University of Calgary" "uni0012" "Jet Propulsion Lab" "uni0013" "University of Manitoba" "uni0014" "United States Geological Survey" "uni0015" "Universität Innsbruck" "uni0016" "Lawrence Berkeley National Laboratory" "uni0017" "University of Michigan–Ann Arbor" "uni0018" "University of Washington" "uni0019" "Agriculture and Agriculture-Food Canada" "uni0020" "Natural Resources Canada" "uni0021" "University of California, Berkeley" "uni0022" "University of Arizona" "uni0023" "University of Quebec at Montreal" "uni0024" "University of Alaska Fairbanks" "uni0025" "Max Pl

In [29]:
inst_code.iloc[:50]

,Institution,code
0,University of Saskatchewan,1
1,University of Waterloo,2
2,McMaster University,3
3,Environment and Climate Change Canada,4
4,University of Guelph,5
5,Global Institute for Water Security,6
6,Wilfrid Laurier University,7
7,University of Toronto,8
8,University of Alberta,9
9,Wageningen University & Research,10
